# Compute tractions using the results from the Greenland traction inversion: 
## <font color=blue>"compute_predicted_strain_rates.ipynb"</font>
#### Jul 25, 2022  <font color=red>(v. testing)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)


1. A G-matrix will be built using "strain" basis functions. 
2. And then the model coefficients will be multiplied to the G-matrix to obtain **${e}_{ij}^{obs}$**.
3. Using the **${e}_{ij}^{obs}$** and the equations in Finzel et al. (2015, GRL), this code computes sigma xx, sigma xy, tau_0. #greek

In [58]:
########Import modules
import numpy as np
import pandas as pd
import sys

import warnings
warnings.filterwarnings('ignore')

In [59]:
# inversion_results = sys.argv[1]
# inversion_results = str(inversion_results)

#model_coef_Ridge_0.16876124757881522.out
#model_coef_Ridge_0.1519911082952933.out

inversion_results = "./inver_results_cm\model_coef_Ridge_0.1519911082952933_1.0.out"
# inversion_results = "./Inversion_results_new_magnitudes\model_coef_Ridge_0.1519911082952933__1.0.out"

In [60]:
np_model_coefs = np.loadtxt(inversion_results)

######## How Many Basis Functions 
HowManyBasisFunctions=np.loadtxt("geometry_info.txt", skiprows=1)
HowManyCell=int(HowManyBasisFunctions[1])
# HowManyCell = 172

# Output files
exx_pred = "exx_pred.xyz"
eyy_pred = "eyy_pred.xyz"
exy_pred = "exy_pred.xyz"

pred1 = "sig_xx_pred.xyz"
pred2 = "tau_0_pred.xyz"
pred3 = "sig_xy_cosy_cosy_pred.xyz"
pred4 = "sig_xy_pred.xyz"
pred5 = "sig_xx_cosy_cosy_pred.xyz"


# `STEP 1:` **BUILD G-Matrix (strain rates), $\bar{\bar{G}}$**

In [61]:
input_sample = "./FILES_basis_functions/average_strain_RECTANGULAR_1_1.out"
sample = np.loadtxt(input_sample)
data_length=len(sample)

In [62]:
lon=sample[:,2]
lat=sample[:,1]

In [63]:
midpoints_on_greenland = pd.read_csv("midpoints_on_greenland.csv")
midpoints_on_greenland

,Unnamed: 0,0,1,2,3,4,5,6,7,8
0,421,422,61.0,-47.0,-0.001589,0.000400,-0.001146,0.0,0.0,0.0
1,422,423,61.0,-45.0,-0.001613,0.000454,-0.001000,0.0,0.0,0.0
2,423,424,61.0,-43.0,-0.001620,0.000488,-0.000865,0.0,0.0,0.0
3,460,461,63.0,-49.0,-0.001231,0.000066,-0.001365,0.0,0.0,0.0
4,461,462,63.0,-47.0,-0.001318,0.000174,-0.001239,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
167,868,869,83.0,-33.0,-0.000212,-0.000334,-0.000305,0.0,0.0,0.0
168,869,870,83.0,-31.0,-0.000241,-0.000307,-0.000320,0.0,0.0,0.0
169,870,871,83.0,-29.0,-0.000270,-0.000279,-0.000332,0.0,0.0,0.0
170,871,872,83.0,-27.0,-0.000300,-0.000250,-0.000342,0.0,0.0,0.0


In [64]:
df_G_exx_resp = pd.DataFrame(index = range(data_length))
df_G_eyy_resp = pd.DataFrame(index = range(data_length)) 
df_G_exy_resp = pd.DataFrame(index = range(data_length)) 

names = ['cell_num','lat','lon','exx_resp','eyy_resp','exy_resp','std_exx_resp','std_eyy_resp','std_exy_resp']
# Make a blank G matrix part related to Boundary Condition on data points

for i in range(1,HowManyCell+1):
    
    inputfile_exx = "./FILES_basis_functions/average_strain_RECTANGULAR_"+str(i)+"_1"+".out" 
    #exx responses
    inputfile_eyy = "./FILES_basis_functions/average_strain_RECTANGULAR_"+str(i)+"_2"+".out" 
    #eyy responses
    inputfile_exy = "./FILES_basis_functions/average_strain_RECTANGULAR_"+str(i)+"_3"+".out" 
    #exy responses 


    df_exx=pd.read_csv(inputfile_exx, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy=pd.read_csv(inputfile_eyy, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy=pd.read_csv(inputfile_exy, header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   

# CHANGE the column names 

    df_exx.columns = names
    df_eyy.columns = names
    df_exy.columns = names
       
    df_G_exx_resp["G_exx_resp_from_exx_"+str(i)] = df_exx.loc[:,['exx_resp']]
    df_G_exx_resp["G_exx_resp_from_eyy_"+str(i)] = df_eyy.loc[:,['exx_resp']]
    df_G_exx_resp["G_exx_resp_from_exy_"+str(i)] = df_exy.loc[:,['exx_resp']]
    
    df_G_eyy_resp["G_eyy_resp_from_exx_"+str(i)] = df_exx.loc[:,['eyy_resp']]
    df_G_eyy_resp["G_eyy_resp_from_eyy_"+str(i)] = df_eyy.loc[:,['eyy_resp']]
    df_G_eyy_resp["G_eyy_resp_from_exy_"+str(i)] = df_exy.loc[:,['eyy_resp']]
    
    df_G_exy_resp["G_exy_resp_from_exx_"+str(i)] = df_exx.loc[:,['exy_resp']]
    df_G_exy_resp["G_exy_resp_from_eyy_"+str(i)] = df_eyy.loc[:,['exy_resp']]
    df_G_exy_resp["G_exy_resp_from_exy_"+str(i)] = df_exy.loc[:,['exy_resp']]


In [65]:
print("DONE")

DONE


In [66]:
print(np_model_coefs.shape)
print(HowManyCell)
print(df_G_exy_resp.shape)
print(df_G_eyy_resp.shape)
print(df_G_exy_resp.shape)

(3000,)
1000
(1000, 3000)
(1000, 3000)
(1000, 3000)


<div class="alert alert-success">
    <b> These are strain rate basis functions ! </b>
</div>

<div class="alert alert-success">
    <b> strain rate basis functions @ model coefficients = predicted strain </b>
</div>

In [67]:
### df G to np G
np_G_exx_resp=df_G_exx_resp.to_numpy()
np_G_eyy_resp=df_G_eyy_resp.to_numpy()
np_G_exy_resp=df_G_exy_resp.to_numpy()

### G*m (prediction for strain)

#when you perturb your model coefficents, obtain the predicted velocity using the new coefficents and run this code to obtain the new strain responses
#that corespond to the new model vectors
#these new strain responses can be used to compute the basal tractions to see how they change
#plot the new velocity field on the surface
#when you reduce the model coefficents, you want to take model prediction add to GPE velocity to get InSAR velocity

np_exx_pred = np_G_exx_resp @ np_model_coefs
np_eyy_pred = np_G_eyy_resp @ np_model_coefs
np_exy_pred = np_G_exy_resp @ np_model_coefs

np_sig_xy = np_exy_pred #strain rate responses to corresponding to basis functions
np_sig_xx = 1/2*(np_exx_pred - np_eyy_pred)
np_tau_0 = 3/2*(np_exx_pred + np_eyy_pred)


np_sig_xy_cosycosy = np_sig_xy*(np.cos(lat*np.pi/180)**2)
np_sig_xx_cosycosy = np_sig_xx*(np.cos(lat*np.pi/180)**2)

### dictionary for  xyz files
dict_exx_pred = {'lon': lon, 'lat': lat, 'exx': np_exx_pred}
dict_eyy_pred = {'lon': lon, 'lat': lat, 'eyy': np_eyy_pred}
dict_exy_pred = {'lon': lon, 'lat': lat, 'exy': np_exy_pred}


dict_sig_xx_pred = {'lon': lon, 'lat': lat, 'sigxx': np_sig_xx}
dict_tau_0_pred = {'lon': lon, 'lat': lat, 'tau0': np_tau_0}
dict_sig_xy_cosycosy_pred = {'lon': lon, 'lat': lat, 'sigxy_cosy_cosy': np_sig_xy_cosycosy}
dict_sig_xy_pred = {'lon': lon, 'lat': lat, 'sigxy': np_sig_xy}
dict_sig_xx_cosycosy_pred = {'lon': lon, 'lat': lat, 'sigxx_cosy_cosy': np_sig_xx_cosycosy}


### dict to df 
df_exx_pred = pd.DataFrame(dict_exx_pred)
df_eyy_pred = pd.DataFrame(dict_eyy_pred)
df_exy_pred = pd.DataFrame(dict_exy_pred)


df_sig_xx_pred = pd.DataFrame(dict_sig_xx_pred)
df_tau_0_pred = pd.DataFrame(dict_tau_0_pred)
df_sig_xy_cosycosy_pred = pd.DataFrame(dict_sig_xy_cosycosy_pred)
df_sig_xy_pred = pd.DataFrame(dict_sig_xy_pred)
df_sig_xx_cosycosy_pred = pd.DataFrame(dict_sig_xx_cosycosy_pred)



### save df

df_exx_pred.to_csv(exx_pred,header=None, index=None, sep=' ')
df_eyy_pred.to_csv(eyy_pred,header=None, index=None, sep=' ')
df_exy_pred.to_csv(exy_pred,header=None, index=None, sep=' ')


df_sig_xx_pred.to_csv(pred1,header=None, index=None, sep=' ')
df_tau_0_pred.to_csv(pred2,header=None, index=None, sep=' ')
df_sig_xy_cosycosy_pred.to_csv(pred3,header=None, index=None, sep=' ')
df_sig_xy_pred.to_csv(pred4,header=None, index=None, sep=' ')
df_sig_xx_cosycosy_pred.to_csv(pred5,header=None, index=None, sep=' ')